In [ ]:
%pip install keras-tuner --upgrade

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
# get dataset
dataset = pd.read_csv("Data/LLVM_AllNumeric.csv")
dataset = dataset.sample(frac=1)
dataset_features = dataset.copy()
dataset_labels = dataset_features.pop('PERF')

In [ ]:
# normalize dataset (MinMaxScale)
features_max = dataset_features.max()
labels_max = dataset_labels.max()
dataset_features /= features_max
dataset_labels /= dataset_labels.max()

In [ ]:
# split dataset train (2/3) test (1/3)
x, x_test, y, y_test = train_test_split(dataset_features, dataset_labels, test_size=0.33)

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    # model.add(keras.layers.Flatten())
    for i in range(hp.Int("num_layers", 1, 11)):
        model.add(
            keras.layers.Dense(
                units=hp.Int(f"units_{i}", min_value=8, max_value=256, step=8),
                activation=hp.Choice("activation", ["relu", "tanh", "sigmoid"])
            )
        )
    if hp.Boolean("dropout"):
        model.add(keras.layers.Dropout(rate=0.25))
    model.add(keras.layers.Dense(1))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-1, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mean_squared_error",
        metrics=[keras.metrics.MeanSquaredError()]
    )
    return model

In [ ]:
tuner = kt.RandomSearch(
    hypermodel = build_model,
    objective="mean_squared_error",
    max_trials=10,
    executions_per_trial=3,
    overwrite=True,
    directory="my_tuner",
    project_name="feature_degradation",
)

In [ ]:
tuner.search_space_summary()

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [ ]:
tuner.search(x_train, y_train, epochs=10, validation_data=(x_val, y_val))

In [89]:
tuner.results_summary()

Results summary
Results in my_tuner/feature_degradation
Showing 10 best trials
Objective(name='mean_squared_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 5
units_0: 96
activation: relu
dropout: False
lr: 0.0039186864355751794
units_1: 184
units_2: 64
units_3: 200
units_4: 8
Score: 0.00037259711340690654
Trial summary
Hyperparameters:
num_layers: 8
units_0: 96
activation: sigmoid
dropout: True
lr: 0.027177000743455176
units_1: 56
units_2: 192
units_3: 184
units_4: 192
units_5: 8
units_6: 8
units_7: 8
Score: 0.005372364850093921
Trial summary
Hyperparameters:
num_layers: 8
units_0: 72
activation: tanh
dropout: True
lr: 0.001461171971454041
units_1: 224
units_2: 64
units_3: 160
units_4: 232
units_5: 200
units_6: 56
units_7: 256
units_8: 224
Score: 0.0058857714757323265
Trial summary
Hyperparameters:
num_layers: 3
units_0: 24
activation: relu
dropout: True
lr: 0.003231343189436178
units_1: 176
units_2: 112
units_3: 152
Score: 0.008238532269994417
Trial summary
Hyperpa

In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
model = build_model(best_hps[0])
history = model.fit(x_train, y_train, batch_size=5, epochs=20, validation_data=(x_val, y_val),)

In [ ]:
keras.utils.plot_model(model, "model.png", show_shapes=True)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=5)
print("test loss, test msqe:", results)


In [ ]:
print("Generate predictions for 3 samples")
test = x_test[:3]
print(test)
predictions = model.predict(test)
print("predictions shape:", predictions.shape)

In [ ]:
print(predictions*labels_max)